# Prediction Model for Candidate Join-NoJoin

In [3]:
import pandas as pd 
import spacy
import en_core_web_sm
import re

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)


# We can load the English NLP pipeline in the following way:
nlp = spacy.load("en_core_web_sm")

# Define a custom method for Data Pre-Processing 
def pre_process(reviewTexts):
    cleanReviewList = []
    # Lowercasing
    text = reviewTexts.lower()
    text = text.replace("'", '')
    # Replacing punctuation in the tokens
    text = re.sub(r"[!\"#$%&'()*+,/:;<=>?[\]^_`{|}~]+", " ", text)
    x = " ".join(token.text for token in nlp(text) if token.text.isalpha())
    return x

In [4]:
import docx
import fitz  # PyMuPDF Module for PDF Parsing
import os

# Definining a custom method for reading PDF Resume using PyMuPDF Library
def getPdfResumeText(file) :
    pdfFileObj = fitz.open(file)
    page_count = pdfFileObj.pageCount

    #print("pdfReader.numPages" ,page_count )
    #print(pdfFileObj.metadata)

    # Extract text from every page on the file
    text = [pdfFileObj.loadPage(i).getText("text") for i in range(page_count)]
    return str(text).replace("\\n", "")

# Defining a custom method for reading docs resume
def getDocResumeText(filename):
    return '\n'.join([para.text for para in docx.Document(filename).paragraphs if para.text.strip()])

# Initialise the path where all the resumes are stored
mypath='E:/AMPBA/Term2/FP/FP-Project/Resumes/' #enter your path here where you saved the resumes
resumeFiles = [os.path.join(mypath, f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
print(resumeFiles)

#Read content of Resumes into a Datarframe 
resumeText = []
for resume in resumeFiles:
    f,ext = os.path.splitext(resume)
    if (ext == ".pdf"):
        resumeText.append(getPdfResumeText(resume))
    elif (ext == ".docx"):
        resumeText.append(getDocResumeText(resume))
    else:
        continue
        
resumes_df = pd.DataFrame(resumeText, columns = ['Resume'])
resumes_df['Id'] = resumes_df.index + 1
resumes_df

['E:/AMPBA/Term2/FP/FP-Project/Resumes/Aayushi Resume.docx', 'E:/AMPBA/Term2/FP/FP-Project/Resumes/Bhaskar_Verma.pdf', 'E:/AMPBA/Term2/FP/FP-Project/Resumes/Manager- Sumit Chadha.doc', 'E:/AMPBA/Term2/FP/FP-Project/Resumes/Manager- Sumit Chadha.pdf', 'E:/AMPBA/Term2/FP/FP-Project/Resumes/Manish_Manohar_Resume.pdf', 'E:/AMPBA/Term2/FP/FP-Project/Resumes/SiddharthResume.pdf', 'E:/AMPBA/Term2/FP/FP-Project/Resumes/VarnikaGupta_finalresume.pdf']


Resume  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [7]:
# Create features list using the input JD 

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import docx

# define method to read the jd document
def getText(filename):
    return '\n'.join([para.text for para in docx.Document(filename).paragraphs if para.text.strip()])

stop_words = stopwords.words("english")

# Reading the jd from the given folder path
jd_text = getText('E:/AMPBA/Term2/FP/FP-Project/JD/jd_java_developer.docx')
jd_temp = set(word_tokenize(pre_process(jd_text)))

# Removing the stop-words from the feature vocabulory
jd_vocab = [token for token in jd_temp if not token in stop_words]
jd_vocab

['noida',
 'location',
 'spring',
 'skill',
 'rest',
 'core',
 'years',
 'exp',
 'webservices',
 'hibernate',
 'java',
 'developers']

In [8]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

# Pre-Processing the Resumes
resumes_df['Resume'] = resumes_df['Resume'].apply(pre_process)  

# Taking Resume content from the resume df for training the model
train_data = resumes_df.Resume 
stop_words = stopwords.words("english")

# Initialize the TfidfVectorizer with maximum features or vocabulories to 50k 
tfidfVectorizer = TfidfVectorizer(stop_words=stop_words, max_features = 1500, vocabulary = jd_vocab)

# Learn and transform train documents by obtaining TFIDF features
vectorised_train_documents = tfidfVectorizer.fit_transform(train_data)
print(vectorised_train_documents.shape)
tfidf_tokens = tfidfVectorizer.get_feature_names()

(6, 12)


In [9]:
df_tfidfvect = pd.DataFrame(data = vectorised_train_documents.toarray(),index = resumes_df.Id, columns = tfidf_tokens)
df_tfidfvect

,noida,location,spring,skill,rest,core,years,exp,webservices,hibernate,java,developers
Id,,,,,,,,,,,,
1,0.366816,0.447328,0.000000,0.000000,0.000000,0.619382,0.000000,0.0,0.00000,0.000000,0.530763,0.0
2,0.042206,0.000000,0.717510,0.000000,0.205882,0.000000,0.035634,0.0,0.05147,0.253239,0.610706,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.00000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.466348,0.568707,0.000000,0.295292,0.098431,0.0,0.00000,0.116587,0.590434,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.503967,0.000000,0.0,0.00000,0.000000,0.863723,0.0


In [10]:
# Convert the input JD also into a vector based on the TFIDFVectorizer Model
vectorised_jd = tfidfVectorizer.transform([jd_text])
print('JD: \n', vectorised_jd.todense())
vectorised_jd.shape

JD: 
 [[0.23867714 0.29106461 0.23867714 0.29106461 0.29106461 0.20150764
  0.20150764 0.38062157 0.29106461 0.23867714 0.34535348 0.38062157]]


(1, 12)

In [12]:
# Calculating the Cosine score for each Resume against the jd to find the similarity

from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(vectorised_jd, vectorised_train_documents)
final_df = pd.DataFrame(sim.T, index = resumes_df.Id, columns = ['JDCosineScore'])
final_df.reset_index(inplace=True)
final_df
final_df.sort_values(by=["JDCosineScore"],ascending=False)

,Id,JDCosineScore
3,4,0.587911
1,2,0.534765
0,1,0.525863
5,6,0.399843
2,3,0.201508
4,5,0.000000
